In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests

## DATA SET MERGED

In this Jupyter notebook, all necessary data from external datasets will be merged with the provided CMU dataset to support the analysis. A frist preprocessing will also be conducted.


In [9]:
# Loading the original dataset 
data_folder ='../data/original_datsets/'

# Headers 
column_names = ['Wiki_ID', 'Movie_ID', 'title','release_date', 'BoxOfficeRevenue', 'Runtime', 'Languages', 'Countries', 'Genres' ]

df = pd.read_csv( data_folder + 'CMU_dataset/movie.metadata.tsv', sep='\t', names=column_names, header=None)

In [7]:
df.head(2)

,Wiki_ID,Movie_ID,title,release_date,BoxOfficeRevenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


The values in the columns "Languages," "Countries," and "Genres" are structured as dictionaries—for example, `{"/m/02h40lc": "English Language"}`, where `"/m/02h40lc"` is the key and `"English Language"` is the value. To simplify future use of these values, they will be converted into lists of strings.

In [8]:
# Extract the values inside the second set of quotes for each key-value pair
def extract_values(_str):
    return re.findall(r': "([^"]+)"', _str)

# Apply the extraction function to the relevant columns
df['Languages'] = df['Languages'].apply(extract_values)
df['Countries'] = df['Countries'].apply(extract_values)
df['Genres'] = df['Genres'].apply(extract_values)

# Displaying the DataFrame to see the result
df.head(2)

,Wiki_ID,Movie_ID,title,release_date,BoxOfficeRevenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"


# IMDb Datasets

The 'title.ratings.tsv' and 'title.akas.tsv' datasets come from the IMDb Non-Commercial Datasets, available at https://developer.imdb.com/non-commercial-datasets/. 

The 'title.ratings.tsv' file contains:
- tconst (string): alphanumeric unique identifier of the title
- averageRating: weighted average of all the individual user ratings
- numVotes: number of votes the title has received

and the 'title.akas.tsv' file contains:

- titleId (string): a tconst, an alphanumeric unique identifier of the title
- ordering (integer): a number to uniquely identify rows for a given titleId
- title (string): the localized title
- region (string):the region for this version of the title
- language (string): the language of the title
- types (array): Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
- attributes (array): Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean): 0: not original title; 1: original title,

These datasets must be merged to connect the 'title' with its corresponding 'averageRating' using the 'titleId'/'tconst'. Once merged, this data will be integrated into the initial dataset. 
IMDb rating data is essential for creating a metric called 'SuccessMetric,' which will be used to classify a movie's success.

In [5]:
#Load IMDb Datasets
df_rating = pd.read_csv(data_folder + 'CMU_dataset/title.ratings.tsv', sep='\t')
df_rating_title = pd.read_csv(data_folder + 'title.akas.tsv', sep='\t')

In [6]:
# Displaying the DataFrame
df_rating.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2096
1,tt0000002,5.6,282


In [7]:
#Displaying the DataFrame
df_rating_title.head(2)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,\N,\N,original,\N,1
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0


In [8]:
df_rating = df_rating.rename(columns={'tconst': 'titleId'})
df_rating.head(2)

,titleId,averageRating,numVotes
0,tt0000001,5.7,2096
1,tt0000002,5.6,282


In [9]:
# Check if there are duplicate movies with the same title ID
# This ensures that each title ID is associated with only one movie entry
df_rating_title[df_rating_title['titleId']=='tt0228333']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
1493874,tt0228333,1,Ghosts of Mars,\N,\N,original,\N,1
1493875,tt0228333,10,Ghosts of Mars,SE,\N,imdbDisplay,\N,0
1493876,tt0228333,11,Ghosts of Mars,US,\N,imdbDisplay,\N,0
1493877,tt0228333,12,Ghosts of Mars,ZA,en,imdbDisplay,\N,0
1493878,tt0228333,13,Fantasmas de Marte,BR,\N,\N,\N,0
1493879,tt0228333,14,Fantasmas de Marte,MX,\N,\N,\N,0
1493880,tt0228333,15,Fantasmas de Marte,PE,\N,imdbDisplay,\N,0
1493881,tt0228333,16,Fantasmas de Marte,US,es,imdbDisplay,\N,0
1493882,tt0228333,17,A Mars szelleme,HU,\N,\N,\N,0
1493883,tt0228333,18,Apeili ston Ari,GR,\N,\N,transliterated title,0


In [10]:
# For each title ID, keep only the original title.
df_rating_title=df_rating_title[df_rating_title['isOriginalTitle'] == 1.0]

In [11]:
# Count the occurrences of each unique value in the title 
value_count = df_rating_title['title'].value_counts()
print(value_count)

title
Episode #1.1                   53140
Episode #1.2                   47989
Episode #1.3                   45430
Episode #1.4                   42165
Episode #1.5                   38656
                               ...  
Astronomy in World Religion        1
Does Hers Like Pumpkin?            1
Sticking to the Script             1
Slung Your Time                    1
Horrid Henry Knows It All          1
Name: count, Length: 5043401, dtype: int64


It seems that there are still duplicates...

In [12]:
# Count the occurrences of each unique value in the titleId
value_count1 = df_rating_title['titleId'].value_counts()
print(value_count1)

titleId
tt3880980     2
tt0000001     1
tt31107355    1
tt31107345    1
tt31107346    1
             ..
tt15067248    1
tt1506725     1
tt15067252    1
tt15067254    1
tt9916880     1
Name: count, Length: 11178939, dtype: int64


Apart from one movie, the duplicates appear to be based on the title rather than the title ID. 
This suggests that they could be different movies with the same title, rather than exact duplicates. 

Let's focus on observing the exception.

In [13]:
df_rating_title[df_rating_title['titleId'] == 'tt3880980']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
38672995,tt3880980,1,Deadly Dreams,\N,\N,original,\N,1
38672996,tt3880980,2,マックのハッスル刑事\tJP\tja\timdbDisplay\t\N\t0\ntt3880...,\N,\N,original,\N,1


It seems to be the same movie in this case, but with different titles considered as each the original title.

In [14]:
#Merged title and rates
df_merged = pd.merge(df_rating, df_rating_title, on='titleId', how='inner')

In [15]:
df_merged

,titleId,averageRating,numVotes,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,5.7,2096,1,Carmencita,\N,\N,original,\N,1
1,tt0000002,5.6,282,1,Le clown et ses chiens,\N,\N,original,\N,1
2,tt0000003,6.5,2114,1,Pauvre Pierrot,\N,\N,original,\N,1
3,tt0000004,5.4,182,1,Un bon bock,\N,\N,original,\N,1
4,tt0000005,6.2,2844,1,Blacksmith Scene,\N,\N,original,\N,1
...,...,...,...,...,...,...,...,...,...,...
1492858,tt9916730,7.0,12,1,6 Gunn,\N,\N,original,\N,1
1492859,tt9916766,7.1,24,1,Episode #10.15,\N,\N,original,\N,1
1492860,tt9916778,7.2,37,1,Escape,\N,\N,original,\N,1
1492861,tt9916840,6.9,11,1,Horrid Henry's Comic Caper,\N,\N,original,\N,1


In [16]:
value_count1 = df_merged['titleId'].value_counts()
print(value_count1)

titleId
tt0000001     1
tt2565978     1
tt2565972     1
tt2565964     1
tt2565962     1
             ..
tt1055718     1
tt1055717     1
tt10557162    1
tt1055716     1
tt9916880     1
Name: count, Length: 1492863, dtype: int64


There are no more duplicates based on the title ID. However, to differentiate between movies with the same title, additional information that varies between these movies is needed.   
The release year will be used for this purpose.

To achieve this, the 'title.basics.tsv'file from the same IMDb webpage as before will be used to add the release year.  
The 'title.basics.tsv' file contains:
- tconst (string): alphanumeric unique identifier of the title
- titleType (string): the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string): the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string): original title, in the original language
- isAdult (boolean): 0: non-adult title; 1: adult title
- startYear (YYYY): represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY): TV Series end year. '\N' for all other title types
- runtimeMinutes: primary runtime of the title, in minutes
- genres (string array): includes up to three genres associated with the title

In [17]:
# Load the new dataset
df_date = pd.read_csv(data_folder + 'IMDB_data/title.basics.tsv', sep='\t')

C:\Users\mehdi\AppData\Local\Temp\ipykernel_24908\2084849319.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_date = pd.read_csv(data_folder + 'title.basics.tsv', sep='\t')


In [18]:
# Merged the two datasets
df_date = df_date.rename(columns={'tconst': 'titleId'})
df_merged_final = pd.merge(df_merged, df_date, on='titleId', how='inner')

In [19]:
# We have now the final dataset that is composed of two datasets that we got from IMDb 
# We used it in order to get the ratings of movies wich is crucial to later define the success of a movie
# The dataset that we now have is composed of : the titleID, the average rating on a scale of 1 to 10,
# the nimber of votes for the rating, the region, the language of the movie, the attributes, some informations on the movie title 
# like the length,type etc..., IsAdult (a bool that tells if the movie is an adult film), the release year, the runtime and the genre.
df_merged_final.head(2)

,titleId,averageRating,numVotes,ordering,title,region,language,types,attributes,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,5.7,2096,1,Carmencita,\N,\N,original,\N,1,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,5.6,282,1,Le clown et ses chiens,\N,\N,original,\N,1,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


The final IMDb dataset will now be merged with the original dataset, utilizing the normalized title and release year to avoid any confusion.

In [20]:
df['Normalized_Title'] = df['title'].str.replace(" ", "").str.lower()
df_merged_final['Normalized_Title'] = df_merged_final['title'].str.replace(" ", "").str.lower()

In [21]:
df['releaseYear'] = df['release_date'].astype(str).str[:4]
df_merged_final['releaseYear'] = df_merged_final['startYear'].astype(str).str[:4]

In [22]:
df_merged_final['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame'], dtype=object)

Examining the 'titleType' category reveals entries labeled as 'videoGame' and 'tvSpecial'.  
Since these are not considered cinema categories, they will be removed from the dataset.

In [23]:
df_merged_final = df_merged_final[~df_merged_final['titleType'].isin(['videoGame', 'tvSpecial'])]

In [24]:
# Merged the original dataset with the treated IMDb dataset
df_IMDb = pd.merge(df, df_merged_final , on=['Normalized_Title', 'releaseYear'], how='inner')

In [25]:
# Check for duplicate titles
print(df_IMDb['title_x'].value_counts())

title_x
Home                       23
Love                       21
Legacy                     19
Macbeth                    18
Alice in Wonderland        18
                           ..
Gangster Wars               1
The Fall of a Nation        1
Needhikku Thalaivanangu     1
Free Ride                   1
Another Nice Mess           1
Name: count, Length: 43429, dtype: int64


In [26]:
#Verify for one duplicate title whether duplicates also exist for the release year.
df_IMDb[df_IMDb['title_x'] == 'Macbeth']

,Wiki_ID,Movie_ID,title_x,release_date,BoxOfficeRevenue,Runtime,Languages,Countries,Genres,Normalized_Title,...,attributes,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
5234,16786364,/m/04062sb,Macbeth,1954-11-28,NaN,103.0,[English Language],"[United States of America, United Kingdom]",[Comedy-drama],macbeth,...,\N,1,tvMovie,Macbeth,Macbeth,0,1954,\N,103,Drama
14910,15097368,/m/03hh969,Macbeth,1909-11-27,NaN,16.0,[Italian Language],[Italy],"[Short Film, Silent film]",macbeth,...,\N,1,short,Macbeth,Macbeth,0,1909,\N,\N,"Drama,Short"
14911,15097368,/m/03hh969,Macbeth,1909-11-27,NaN,16.0,[Italian Language],[Italy],"[Short Film, Silent film]",macbeth,...,\N,1,short,Macbeth,Macbeth,0,1909,\N,\N,"Drama,Short"
14912,15097255,/m/03hh92g,Macbeth,1909-12-03,NaN,NaN,[],[France],[Silent film],macbeth,...,\N,1,short,Macbeth,Macbeth,0,1909,\N,\N,"Drama,Short"
14913,15097255,/m/03hh92g,Macbeth,1909-12-03,NaN,NaN,[],[France],[Silent film],macbeth,...,\N,1,short,Macbeth,Macbeth,0,1909,\N,\N,"Drama,Short"
20326,15097470,/m/03hh99d,Macbeth,1913,NaN,NaN,[],[Germany],"[Silent film, Indie]",macbeth,...,\N,1,movie,Macbeth,Macbeth,0,1913,\N,50,"Crime,Drama"
20974,6652508,/m/0gg5jd,Macbeth,2006-09-21,NaN,109.0,[English Language],"[Australia, New Zealand]","[Crime Fiction, Thriller, Indie, World cinema,...",macbeth,...,\N,1,movie,Macbeth,Macbeth,0,2006,\N,109,"Crime,Drama,Fantasy"
20975,6652508,/m/0gg5jd,Macbeth,2006-09-21,NaN,109.0,[English Language],"[Australia, New Zealand]","[Crime Fiction, Thriller, Indie, World cinema,...",macbeth,...,\N,1,movie,Macbeth,Macbeth,0,2006,\N,118,Drama
20976,6652508,/m/0gg5jd,Macbeth,2006-09-21,NaN,109.0,[English Language],"[Australia, New Zealand]","[Crime Fiction, Thriller, Indie, World cinema,...",macbeth,...,\N,1,tvMovie,Macbeth,Macbeth,0,2006,\N,156,Music
24345,1966825,/m/069mfr,Macbeth,1948-10-01,NaN,107.0,[English Language],[United States of America],"[Costume drama, Drama]",macbeth,...,\N,1,movie,Macbeth,Macbeth,0,1948,\N,107,"Drama,History,War"


Duplicate entries for year and title were observed, corresponding to different ratings, possibly from different webpages for the same movie. To address this, a weighted average will be calculated.

In [27]:
# Compute the average rating
df_IMDb['weighted_rating'] = df_IMDb['averageRating'] * df_IMDb['numVotes']

df_IMDb_f = df_IMDb.groupby(['Normalized_Title', 'releaseYear']).agg(
    Wiki_ID = ('Wiki_ID', 'first'),
    Movie_ID = ('Movie_ID', 'first'),
    title=('title_x', 'first'),  
    totRating=('weighted_rating', 'sum'),
    numVotes=('numVotes', 'sum'),
    release_date = ('release_date', 'first'),
    Runtime =  ('Runtime', 'first'),
    BoxOfficeRevenue = ('BoxOfficeRevenue', 'first'),
    Languages = ('Languages', 'first' ),
    Countries = ('Countries', 'first'),
    Genres=('Genres', 'first')

).reset_index()

df_IMDb_f['Rating'] = df_IMDb_f['totRating'] / df_IMDb_f['numVotes']

df_IMDb_f= df_IMDb_f.drop(columns=['totRating'])

In [28]:
df_IMDb_f.head(2)

,Normalized_Title,releaseYear,Wiki_ID,Movie_ID,title,numVotes,release_date,Runtime,BoxOfficeRevenue,Languages,Countries,Genres,Rating
0,#1cheerleadercamp,2010,30332673,/m/0crs0hx,#1 Cheerleader Camp,3424,2010,90.0,NaN,[],[United States of America],"[Sports, Sex comedy, Comedy film, Comedy, Teen]",3.7
1,$,1971,4213160,/m/0bq8q8,$,2948,1971-12-17,119.0,NaN,[English Language],[United States of America],"[Crime Fiction, Heist, Action/Adventure, Thril...",6.3


In [29]:
# Save the DataFrame to a new CSV file
merge_directory = '../data/merged_datasets/'
df_IMDb_f['year'] = df_IMDb_f['release_date'].astype(str).str[:4]
df_IMDb_f.to_csv(merge_directory + 'IMDb.csv', index=False)


In [30]:
df_IMDb_f.head(2)

,Normalized_Title,releaseYear,Wiki_ID,Movie_ID,title,numVotes,release_date,Runtime,BoxOfficeRevenue,Languages,Countries,Genres,Rating,year
0,#1cheerleadercamp,2010,30332673,/m/0crs0hx,#1 Cheerleader Camp,3424,2010,90.0,NaN,[],[United States of America],"[Sports, Sex comedy, Comedy film, Comedy, Teen]",3.7,2010
1,$,1971,4213160,/m/0bq8q8,$,2948,1971-12-17,119.0,NaN,[English Language],[United States of America],"[Crime Fiction, Heist, Action/Adventure, Thril...",6.3,1971


# BechdelTest Dataset

The 'df_bechdeltest' DataFrame is sourced from http://bechdeltest.com/api/v1/getAllMovies Web page, which provides data on movies rated based on the Bechdel Test.

The Bechdel Test is a measure of the representation of women in films. A movie passes the test if it meets three criteria:
1. It features at least two named female characters,
2. who talk to each other,
3. about something other than a man.

While passing the Bechdel Test doesn’t fully capture the complexity of female representation, it offers a useful baseline for understanding gender dynamics in cinema.
This dataset will be used to observe whether the roles of women in films are significant and to analyze how this representation has evolved over the years.


In [31]:
url = 'http://bechdeltest.com/api/v1/getAllMovies'
# The data in this dataset is presented as such : 
# imdbid : the imdb id of the movie
# title
# rating : the result of the bechdel test, 1 if it has to have at least two [named] women in it, 2 if they talk to each other and 3 if it's about 
# something besides a man
# year : year of release of the movie
# id

response = requests.get(url)

if response.status_code == 200:
    print('Request was succesful')
    movie_data = response.json()
    df_bechdel = pd.DataFrame(movie_data)
    print(df_bechdel.sample())

else:
    print('Request failed')

Request was succesful
      rating    id   imdbid  year       title
5501       3  6577  0799949  2007  Epic Movie


In [32]:
df_bechdel.sample(2)

,rating,id,imdbid,year,title
1865,2,303,0077975,1978,Animal House
2842,3,9810,0102397,1991,"Manta, Manta"


In [33]:
df_bechdel['Normalized_Title'] = df_bechdel['title'].str.replace(" ", "").str.lower()

In [34]:
df['year'] = df['release_date'].astype(str).str[:4]

In [35]:
df_bechdel['year'] = df_bechdel['year'].astype(str)

In [36]:
# Merge the original dataset with the Bechdel Test rating dataset 
df_bech_merged = pd.merge(df_bechdel,df , on=['Normalized_Title', 'year'], how='inner')

In [37]:
# Save the DataFrame to a new CSV file
df_bech_merged.to_csv('bechdeltest.csv', index=False)

The IMDb rating dataset is merged with the Bechdel Test dataset to combine information about movie ratings with data on gender representation in films. 

This allows for a more comprehensive analysis of both the quality of the movies and their portrayal of female characters, based on the Bechdel Test criteria.


In [38]:
df_imdb_bech = pd.read_csv(data_folder + 'IMDb.csv')

In [39]:
df_imdb_bech = df_imdb_bech.rename(columns={'Year': 'year'})
df_imdb_bech.head(2)

,Normalized_Title,releaseYear,Wiki_ID,Movie_ID,title,numVotes,release_date,Runtime,BoxOfficeRevenue,Languages,Countries,Genres,Rating,year
0,#1cheerleadercamp,2010,30332673,/m/0crs0hx,#1 Cheerleader Camp,3424,2010,90.0,NaN,[],['United States of America'],"['Sports', 'Sex comedy', 'Comedy film', 'Comed...",3.7,2010
1,$,1971,4213160,/m/0bq8q8,$,2948,1971-12-17,119.0,NaN,['English Language'],['United States of America'],"['Crime Fiction', 'Heist', 'Action/Adventure',...",6.3,1971


In [40]:
df_imdb_bech['year'] = df_imdb_bech['year'].astype(str)

In [41]:
# Merge the IMDb dataset with the Bechdel Test rating dataset 
df_bech_imdb_merged = pd.merge(df_bechdel,df_imdb_bech, on=['Normalized_Title', 'year'], how='inner')

In [42]:
# Rename and remove some columns to make the DataFrame clearer
df_bech_imdb_merged = df_bech_imdb_merged.rename(columns={'rating': 'ratingBechtest'})
df_bech_imdb_merged = df_bech_imdb_merged.loc[:, ~df_bech_imdb_merged.columns.str.endswith('_x')]
df_bech_imdb_merged.columns = df_bech_imdb_merged.columns.str.replace('_y$', '', regex=True)

In [43]:
# Save the DataFrame to a new CSV file
df_bech_imdb_merged.to_csv(merge_directory +'whole_data.csv', index=False)

# Adding budget to the dataset

In order to get the information on the budget of movie, we searched for a dataset online containing this information.
We found on this website : https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset a dataset that might be a good fit. This dataset is a file called movies_metadata_TMDB.csv and its collumns contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

In [44]:
budget_data = pd.read_csv(data_folder+'movies_metadata_TMDB.csv')

# We can drop duplicates by Wiki_ID in our original dataset since it will bias the results
df= df.drop_duplicates(subset='Wiki_ID')

df.shape

C:\Users\mehdi\AppData\Local\Temp\ipykernel_24908\2301449931.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  budget_data = pd.read_csv(data_folder+'movies_metadata_TMDB.csv')


(81741, 12)

In [45]:
# remove the lines where the budget is equal to 0
budget_data['budget'] = pd.to_numeric(budget_data['budget'], errors='coerce')
budget_data_clean = budget_data[budget_data['budget'].notna() & (budget_data['budget'] != 0)]

# removing the duplicates from this dataset also by the imdb_ID
budget_data_clean = budget_data_clean.drop_duplicates(subset='imdb_id')

budget_data_clean.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
3,False,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
5,False,NaN,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0


In [47]:
# We want first of all to ensure that the release_date column is in string format for both datasets
df['release_date'] = df['release_date'].astype(str)
budget_data_clean['release_date'] = budget_data_clean['release_date'].astype(str)

# Extracting the year part safely, cleaning it for the merge
df['year'] = df['release_date'].str[:4]
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')

budget_data_clean['year'] = budget_data_clean['release_date'].str[:4]
budget_data_clean['year'] = pd.to_numeric(budget_data_clean['year'], errors='coerce').astype('Int64')

In [48]:
# now we want to merge the two data sets by the normalized title of the movie and the normalized release year, 
# then add the budget + vote_average to the movie dataset


df['Normalized_Title'] = df['title'].str.replace(" ", "").str.lower()
budget_data_clean['Normalized_Title'] = budget_data_clean['title'].str.replace(" ", "").str.lower()


merged_data_budget = df.merge(budget_data_clean[['Normalized_Title', 'year', 'budget','vote_average']], 
                              on=['Normalized_Title', 'year'], 
                              how='inner')


merged_data_budget.shape


(5832, 14)

In [50]:
# Now we use the IMDb dataset to get the ratings 
# since we also have ratings in the budget dataset, we take the mean of both of them 

df_IMDb_f.rename(columns={'BoxOfficeRevenue':'BoxOfficeRevenueImdb'}, inplace=True)
df_IMDb_f['year'] = df_IMDb_f['year'].astype(int)  # Example conversion



merged_data_imdb = merged_data_budget.merge(df_IMDb_f[['Normalized_Title','BoxOfficeRevenueImdb','Rating','year']],
                                            on=['Normalized_Title','year'], 
                                            how='inner')

merged_data_imdb.sample(5)

,Wiki_ID,Movie_ID,title,release_date,BoxOfficeRevenue,Runtime,Languages,Countries,Genres,Normalized_Title,releaseYear,year,budget,vote_average,BoxOfficeRevenueImdb,Rating
258,1056686,/m/0426cx,Cat People,1942,4000000.0,73.0,"[Czech Language, English Language, Serbian lan...",[United States of America],"[Thriller, Romance Film, Horror, Black-and-whi...",catpeople,1942,1942,134000.0,6.9,4000000.0,7.2
5265,31458474,/m/0gkxx_0,The Tall Man,2012-03-12,NaN,106.0,[English Language],"[France, United States of America, Canada]","[Thriller, Drama, Crime, Mystery]",thetallman,2012,2012,18200000.0,5.7,NaN,5.9
2802,553101,/m/02p8gn,The Fly,1958,NaN,93.0,[English Language],[United States of America],"[Cult, Science Fiction, Horror]",thefly,1958,1958,700000.0,6.9,NaN,7.1
385,736094,/m/036ytw,Moonwalker,1988,NaN,93.0,[English Language],[United States of America],"[Parody, Music, Musical, Concert film]",moonwalker,1988,1988,22000000.0,6.0,NaN,6.1
4723,509987,/m/02jygq,Heaven,2002-02-06,NaN,93.0,"[Italian Language, English Language]","[United States of America, France, Italy, Unit...","[Thriller, Crime Fiction, Indie, World cinema,...",heaven,2002,2002,11000000.0,6.8,NaN,6.9


In [51]:
# renaming 
merged_data_imdb_clean = merged_data_imdb

# now for all the lines where there is a value for BoxOfficeRevenueImdb and not for BoxOfficeRevenue we will copy the value from BoxOfficeRevenueImdb to BoxOfficeRevenue and vice versa
merged_data_imdb_clean['BoxOfficeRevenue'] = merged_data_imdb_clean['BoxOfficeRevenue'].fillna(merged_data_imdb_clean['BoxOfficeRevenueImdb'])
merged_data_imdb_clean['BoxOfficeRevenueImdb'] = merged_data_imdb_clean['BoxOfficeRevenueImdb'].fillna(merged_data_imdb_clean['BoxOfficeRevenue'])

# now for all the lines where there is a value for BoxOfficeRevenueImdb and for BoxOfficeRevenue we will calculate the difference in percentage between the two values and put it in a new column called BoxOfficeRevenueDifference
def percentage_difference(X, Y):                                           

    return np.abs((X-Y))/np.maximum(X,Y) * 100

merged_data_imdb_clean['BoxOfficeRevenueDifference'] = percentage_difference(merged_data_imdb_clean['BoxOfficeRevenue'], merged_data_imdb_clean['BoxOfficeRevenueImdb'])

# print the mean of the BoxOfficeRevenueDifference column
# we do this step to ensure that there is no big difference between the revenues in both datasets 

print(merged_data_imdb_clean['BoxOfficeRevenueDifference'].mean())


merged_data_imdb_clean.sample(5)

0.027667015674755276


,Wiki_ID,Movie_ID,title,release_date,BoxOfficeRevenue,Runtime,Languages,Countries,Genres,Normalized_Title,releaseYear,year,budget,vote_average,BoxOfficeRevenueImdb,Rating,BoxOfficeRevenueDifference
985,27872044,/m/0cc703f,Trespass,2011-09-14,9612469.0,91.0,[English Language],[United States of America],"[Crime Fiction, Thriller, Drama, Psychological...",trespass,2011,2011,36000000.0,5.4,9612469.0,5.3,0.0
2359,1037120,/m/040f3x,The Hand That Rocks the Cradle,1992-01-10,88036683.0,110.0,[English Language],[United States of America],"[Thriller, Drama, Psychological thriller]",thehandthatrocksthecradle,1992,1992,11700000.0,6.5,88036683.0,6.7,0.0
4406,20320460,/m/04zzt8k,Black Dynamite,2009-10-16,296557.0,83.0,[English Language],"[United States of America, United Kingdom]","[Parody, Adventure, Action Comedy, Action/Adve...",blackdynamite,2009,2009,2900000.0,7.3,296557.0,7.4,0.0
5,261237,/m/01mrrd,The Gods Must Be Crazy,1980,34331783.0,109.0,"[Afrikaans Language, English Language]",[South Africa],"[Adventure, Action/Adventure, Indie, World cin...",thegodsmustbecrazy,1980,1980,5000000.0,7.1,34331783.0,7.3,0.0
1983,189088,/m/019wds,A Perfect Murder,1998-06-05,128038368.0,108.0,[English Language],[United States of America],"[Thriller, Crime Thriller, Suspense]",aperfectmurder,1998,1998,60000000.0,6.3,128038368.0,6.6,0.0


In [52]:
# Removing again every duplicate line with the help of the Wiki_ID column
merged_data_imdb_clean = merged_data_imdb_clean.drop_duplicates(subset='Wiki_ID')


# adding a new column called Net_revenue that is  basically the difference between the BoxOfficeRevenue and the budget
merged_data_imdb_clean['Net_revenue'] = merged_data_imdb_clean['BoxOfficeRevenue'] - merged_data_imdb_clean['budget']


merged_data_imdb_clean.sample(5)

C:\Users\mehdi\AppData\Local\Temp\ipykernel_24908\1486757544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_imdb_clean['Net_revenue'] = merged_data_imdb_clean['BoxOfficeRevenue'] - merged_data_imdb_clean['budget']


,Wiki_ID,Movie_ID,title,release_date,BoxOfficeRevenue,Runtime,Languages,Countries,Genres,Normalized_Title,releaseYear,year,budget,vote_average,BoxOfficeRevenueImdb,Rating,BoxOfficeRevenueDifference,Net_revenue
4967,2973132,/m/08g_jw,The Namesake,2006-09-02,20138836.0,122.0,"[Hindi Language, French Language, Bengali Lang...","[United States of America, India]","[Family Drama, Drama, Indie, Film adaptation]",thenamesake,2006,2006,12000000.0,6.8,20138836.0,7.5,0.0,8138836.0
582,2810158,/m/0844ht,Barnyard,2006-08-04,108000000.0,90.0,[English Language],"[United States of America, Germany]","[Computer Animation, Family Film, Children's/F...",barnyard,2006,2006,51000000.0,5.3,108000000.0,5.6,0.0,57000000.0
4498,206879,/m/01dc8g,Star Trek III: The Search for Spock,1984-06-01,87000000.0,105.0,[English Language],[United States of America],"[Science Fiction, Family Film, Adventure]",startrekiii:thesearchforspock,1984,1984,18000000.0,6.4,87000000.0,6.6,0.0,69000000.0
3008,1102966,/m/046639,Killer's Kiss,1955-09-28,NaN,63.0,[English Language],[United States of America],"[Crime Fiction, Thriller, Gangster Film, Black...",killer'skiss,1955,1955,75000.0,6.2,NaN,6.6,NaN,NaN
2169,77609,/m/0kb1g,Sergeant York,1941,4000000.0,134.0,[English Language],[United States of America],"[Biography, Costume drama, Biopic [feature], B...",sergeantyork,1941,1941,1400000.0,7.1,4000000.0,7.7,0.0,2600000.0


In [53]:

# removing a few useless columns for the final dataframe.
merged_data_imdb_clean.drop(columns=['vote_average', 'Rating'], inplace=True)
merged_data_imdb_clean.drop(columns=['BoxOfficeRevenueImdb', 'Normalized_Title', 'BoxOfficeRevenueDifference'], inplace=True)


merged_data_imdb_clean.to_csv('movie_data_successmetric_rating.csv', index=False)

C:\Users\mehdi\AppData\Local\Temp\ipykernel_24908\3680236095.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_imdb_clean.drop(columns=['vote_average', 'Rating'], inplace=True)
C:\Users\mehdi\AppData\Local\Temp\ipykernel_24908\3680236095.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_imdb_clean.drop(columns=['BoxOfficeRevenueImdb', 'Normalized_Title', 'BoxOfficeRevenueDifference'], inplace=True)


# Adding script share data